In [35]:
import glob
import numpy as np
import pandas as pd

def calculate(fname):
    df=pd.read_csv(fname)
    df=df[~(df.videoBufferLength==0).cumprod().astype("bool")]
    arr=df.videoBufferLength.values
    for i in range(1,len(arr)):
        if arr[i]==0 and arr[i-1]>=0.1:
            arr[i]=arr[i-1]-0.1
    df['videoBufferLength']=arr
    print(fname)
    print(df.shape,df[~(df.videoBufferLength==0)].shape)
    if df[~(df.videoBufferLength==0)].shape[0]<3000:
        take=df[~(df.videoBufferLength==0)].shape[0]
    else:
        take=3000
    df=df.loc[:df[~(df.videoBufferLength==0)].reset_index().iloc[take-1]['index']]
    f_df=df[df.videoBufferLength>0]

    #Calculate QoE, BuffTime, BitRate
    buffTime=(df.videoBufferLength==0).sum()/10 #second

    bitRate=f_df.videoBitrate.mean()
     # f_df.videoBitrate.min()
     # print(df.shape,df.videoBitrate.max(),df.videoBitrate.min(),np.log10(df.videoBitrate/df.videoBitrate.min()).sum())

    QoE=\
    np.log10(f_df.videoBitrate/200).sum()-\
    2.66*buffTime-\
    np.abs(np.log10(df.videoBitrate.values[1:])-np.log10(df.videoBitrate.values[:-1])).sum()
    diff=np.abs(np.log10(df.videoBitrate.values[1:])-np.log10(df.videoBitrate.values[:-1])).sum()
    return [fname,buffTime, bitRate, QoE, diff]
#/media/lalan/StudyMaterial/WorkingD/Result/SA/Test/VS_Data/PacketLoss/Throughput/Reno/5%
#/media/lalan/StudyMaterial/WorkingD/Result/SA/Test/VS_Data/PacketLoss/Throughput/BBR/3%/*
#/media/lalan/StudyMaterial/WorkingD/Result/SA/VideoStreamingLog/SERVER_LOG/Throu/Reno/Packet Loss/5%
files=glob.glob("/media/lalan/StudyMaterial/WorkingD/MPD/ClientLog/Reno/Throu/Delay/20M/300ms/*")

stat=[]
for f in files:
    stat.append(calculate(f))

df_stat=pd.DataFrame(stat,columns=['fname','buffTime','bitRate','QoE','Diff'])
print(df_stat)
print(df_stat.drop(columns=['fname']).mean())

/media/lalan/StudyMaterial/WorkingD/MPD/ClientLog/Reno/Throu/Delay/20M/300ms/Reno_T_20M_300ms_1.csv
(3019, 36) (3019, 36)
/media/lalan/StudyMaterial/WorkingD/MPD/ClientLog/Reno/Throu/Delay/20M/300ms/Reno_T_20M_300ms_2.csv
(3033, 36) (3033, 36)
/media/lalan/StudyMaterial/WorkingD/MPD/ClientLog/Reno/Throu/Delay/20M/300ms/Reno_T_20M_300ms_3.csv
(3810, 36) (3810, 36)
/media/lalan/StudyMaterial/WorkingD/MPD/ClientLog/Reno/Throu/Delay/20M/300ms/Reno_T_20M_300ms_4.csv
(3064, 36) (3064, 36)
/media/lalan/StudyMaterial/WorkingD/MPD/ClientLog/Reno/Throu/Delay/20M/300ms/Reno_T_20M_300ms_5.csv
(3052, 36) (3052, 36)
                                               fname  buffTime      bitRate   
0  /media/lalan/StudyMaterial/WorkingD/MPD/Client...       0.0  1057.216667  \
1  /media/lalan/StudyMaterial/WorkingD/MPD/Client...       0.0  1057.858000   
2  /media/lalan/StudyMaterial/WorkingD/MPD/Client...       0.0  1057.825667   
3  /media/lalan/StudyMaterial/WorkingD/MPD/Client...       0.0  1266.63833

In [1]:
# Network Delay
# 100ms BBR: Throughput Bitrate variation
# 1M
[772.00, 673.17, 504.00] # BBR
[772.00, 544.31, 517.21] # BBRv2
[634.45, 722.59, 505.52] # Cubic
[613.58, 576.53, 514.37] # Reno

# 10M
[3814.83, 1892.42, 954.84] # BBR
[3843.00, 2345.41, 1387.32] # BBRv2
[3842.88, 1811.44, 1177.88] # Cubic
[3843.00, 2103.08, 1045.48] # Reno

# 20M
[3843.00, 2287.62, 1426.50] # BBR
[3843.00, 2296.75, 1397.43] # BBRv2
[3843.00, 2108.20, 1126.76] # Cubic
[3843.00, 2255.47, 1099.28] # Reno

In [ ]:
# Buffer Size:
# Reno 1M 10K(Bola)
bitrate = [369.43, 677.25, 772.00]
QoE = [747.67, 1544.57, 1759.76]

buffTime      0.000000
bitRate     369.437000
QoE         747.677734
Diff          3.639598

# 100K
buffTime       0.000000
bitRate      677.253800
QoE         1544.574869
Diff           3.145719

# 1M
buffTime       0.000000
bitRate      772.000000
QoE         1759.761914
Diff           0.000000

# Throughput 1M
bitrate = [365.84, 613.58, 772.00]
QoE = [739.61, 1393.94, 1759.76]

# 10K
buffTime      0.000000
bitRate     365.847200
QoE         739.616958
Diff          2.888901

#100K
buffTime       0.000000
bitRate      613.589667
QoE         1393.946318
Diff           4.932479

#1M
buffTime       0.000000
bitRate      772.000000
QoE         1759.761914
Diff           0.000000

# 10M (Bola)
bitrate = [384.81, 3843.00, 3843.00]
QoE = [839.01, 3850.92, 3850.92]

# 10K
buffTime      0.000000
bitRate     384.812667
QoE         839.011422
Diff          1.246933

# 100K
buffTime       0.000000
bitRate     3843.000000
QoE         3850.921166
Diff           0.000000

#1M
buffTime       0.000000
bitRate     3843.000000
QoE         3850.921166
Diff           0.000000

#Througput 10M
bitrate = [379.28, 3843.00, 3843.00]
QoE = [813.35, 3850.92, 3850.92]

# 10K
buffTime      0.000000
bitRate     379.280000
QoE         813.356106
Diff          1.557147

#100K
buffTime       0.000000
bitRate     3843.000000
QoE         3850.921166
Diff           0.000000

#1M
buffTime       0.000000
bitRate     3843.000000
QoE         3850.921166
Diff           0.000000

# 20M (Bola)
bitrate = [378.90, 3843.00, 3843.00]
QoE = [809.80, 3850.92, 3850.92]

# 10K
buffTime      0.000000
bitRate     378.906867
QoE         809.807655
Diff          1.677111

#100K
buffTime       0.000000
bitRate     3843.000000
QoE         3850.921166
Diff           0.000000

#1M
buffTime       0.000000
bitRate     3843.000000
QoE         3850.921166
Diff           0.000000

# Throughput 20M
bitrate = [382.13, 3843.00, 3843.00]
QoE = [827.68, 3850.92, 3850.92]

# 10K
buffTime      0.000000
bitRate     382.136400
QoE         827.686224
Diff          1.387507

# 100K
buffTime       0.000000
bitRate     3843.000000
QoE         3850.921166
Diff           0.000000

# 1M
buffTime       0.000000
bitRate     3843.000000
QoE         3850.921166
Diff           0.000000

In [ ]:
# Buffer Size:
# Cubic 1M 10K(Bola)
bitrate = [387.43, 676.49, 772.00]
QoE = [861.06, 1542.61, 1759.76]

buffTime      0.000000
bitRate     387.437467
QoE         861.062067
Diff          0.175717

#100K
buffTime       0.000000
bitRate      676.495333
QoE         1542.618997
Diff           3.180863

#1M
buffTime       0.000000
bitRate      772.000000
QoE         1759.761914
Diff           0.000000

# Throughput 1M
bitrate = [382.31, 634.45, 772.00]
QoE = [837.08, 1451.05, 1759.76]

# 10K
buffTime      0.000000
bitRate     382.316533
QoE         837.083862
Diff          0.380501

#100K
buffTime       0.000000
bitRate      634.451333
QoE         1451.059077
Diff           4.245992

#1M
buffTime       0.000000
bitRate      772.000000
QoE         1759.761914
Diff           0.000000

# 10M (Bola)
bitrate = [372.12, 3843.00, 3843.00]
QoE = [789.14, 3850.92, 3850.92]

# 10K
buffTime      0.000000
bitRate     372.126133
QoE         789.148061
Diff          0.860356

# 100K
buffTime       0.000000
bitRate     3843.000000
QoE         3850.921166
Diff           0.000000

# 1M
buffTime       0.000000
bitRate     3843.000000
QoE         3850.921166
Diff           0.000000

# Throughput 10M
bitarte = [384.46, 3842.88, 3843.00]
QoE = [845.79, 3850.81, 3850.92]

# 10K
buffTime      0.000000
bitRate     384.465267
QoE         845.798926
Diff          0.415644

#100K
buffTime       0.000000
bitRate     3842.880250
QoE         3850.819479
Diff           0.050844

#1M
buffTime       0.000000
bitRate     3843.000000
QoE         3850.921166
Diff           0.000000

# 20M (Bola) 10K
bitarte = [375.29, 3843.00, 3843.00]
QoE = [801.19, 3850.92, 3850.92]

buffTime      0.000000
bitRate     375.291333
QoE         801.196205
Diff          0.810679

# 100K
buffTime       0.000000
bitRate     3843.000000
QoE         3850.921166
Diff           0.000000

#1M
buffTime       0.000000
bitRate     3843.000000
QoE         3850.921166
Diff           0.000000

# Throughput 20M

bitrate = [360.16, 3843.00, 3843.00]
QoE = [732.74, 3850.92, 3850.92]

# Throughut 10K
buffTime      0.000000
bitRate     360.160133
QoE         732.740286
Diff          1.460174

# 100K
buffTime       0.000000
bitRate     3843.000000
QoE         3850.921166
Diff           0.000000

#1M
buffTime       0.000000
bitRate     3843.000000
QoE         3850.921166
Diff           0.000000

In [ ]:
# Buffer Size:
# BBRv2 1M 10K(Bola)
bitrate = [557.58, 724.26, 772.00]
QoE = [1322.29, 1651.67, 1759.76]

buffTime       0.000000
bitRate      557.586267
QoE         1322.292548
Diff           0.576828

# 100K
buffTime       0.000000
bitRate      724.266200
QoE         1651.677178
Diff           1.385648

# 1M
buffTime       0.000000
bitRate      772.000000
QoE         1759.761914
Diff           0.000000

# Throught 1M
bitarte = [542.10, 772.00, 772.00]
QoE = [1281.08, 1759.76, 1759.76]

# 10K
buffTime       0.000000
bitRate      542.107667
QoE         1281.082364
Diff           1.372378

#100K
buffTime       0.000000
bitRate      772.000000
QoE         1759.761914
Diff           0.000000

#1M
buffTime       0.000000
bitRate      772.000000
QoE         1759.761914
Diff           0.000000

# (Bola) 10M 10K
bitrate = [511.48, 3843.00, 3843.00]
QoE = [1182.42, 3850.92, 3850.92]

buffTime       0.000000
bitRate      511.485000
QoE         1182.422196
Diff           1.910429

# 100K
buffTime       0.000000
bitRate     3843.000000
QoE         3850.921166
Diff           0.000000

#1M
buffTime       0.000000
bitRate     3843.000000
QoE         3850.921166
Diff           0.000000

# Throughput 10M
bitrate = [463.78, 3843.00, 3843.00]
QoE = [1053.28, 3850.92, 3850.92]

# 10K
buffTime       0.000000
bitRate      463.787867
QoE         1053.286058
Diff           3.465196

#100K
buffTime       0.000000
bitRate     3843.000000
QoE         3850.921166
Diff           0.000000

#1M
buffTime       0.000000
bitRate     3843.000000
QoE         3850.921166
Diff           0.000000

# 20M (Bola) 10K
bitarte = [462.59, 3808.06, 3843.00]
QoE = [1028.40, 3830.76, 3850.92]

buffTime       0.000000
bitRate      462.590000
QoE         1028.400875
Diff           3.723885

# 100K
buffTime       0.000000
bitRate     3808.068200
QoE         3830.767316
Diff           0.278821

# 1M
buffTime       0.000000
bitRate     3843.000000
QoE         3850.921166
Diff           0.000000

# Throughput 20M
bitrate = [442.86, 3843.00, 3843.00]
QoE = [995.75, 3850.92, 3850.92]

# Throughput 10K
buffTime      0.000000
bitRate     442.867067
QoE         995.755773
Diff          3.491883

# 100K
buffTime       0.000000
bitRate     3843.000000
QoE         3850.921166
Diff           0.000000

# 1M
buffTime       0.000000
bitRate     3843.000000
QoE         3850.921166
Diff           0.000000

In [ ]:
# Buffer Size:
# BBR 1M 10K(Bola)
bitrate = [563.81, 772.00, 772.00]
QoE = [1336.58, 1759.76, 1759.76]

buffTime       0.000000
bitRate      563.813733
QoE         1336.580033
Diff           0.591361

# 100K
buffTime       0.000000
bitRate      772.000000
QoE         1759.761914
Diff           0.000000

# 1M
buffTime       0.000000
bitRate      772.000000
QoE         1759.761914
Diff           0.000000


# Throughput 1M
bitrate = [494.37, 772.00, 772.00]
QoE = [1141.85, 1759.76, 1759.76]

# 10K
buffTime       0.000000
bitRate      494.371533
QoE         1141.852763
Diff           2.247329

# 100K
buffTime       0.000000
bitRate      772.000000
QoE         1759.761914
Diff           0.000000

#1M
buffTime       0.000000
bitRate      772.000000
QoE         1759.761914
Diff           0.000000



# 10M (Bola)
bitrate = [553.01, 3814.83, 3843.00]
QoE = [1310.17, 3838.80, 3850.92]

# 10K
buffTime       0.000000
bitRate      553.018600
QoE         1310.172834
Diff           0.717402

# 100K
bitRate     3814.834800
QoE         3838.800031
Diff           0.162700

buffTime       0.000000
bitRate      959.695600
QoE         1916.942925
Diff           9.297112

#1M
buffTime       0.000000
bitRate     3843.000000
QoE         3850.921166
Diff           0.326597

#Throu 10M
bitarte = [580.00, 3814.83, 3843.00]
QoE = [1387.19, 3838.80, 3177.52]

# 10K
buffTime       0.000000
bitRate      580.000000
QoE         1387.193994
Diff           0.000000

#100K
bitRate     3814.834800
QoE         3838.800031
Diff           0.162700

buffTime       0.000000
bitRate     1039.763933
QoE         2033.649709
Diff           7.274815

# 1M
buffTime       0.000000
bitRate     3843.000000
QoE         3177.523418
Diff           0.468286

# 20M (Bola) 10K
bitarte = [537.99, 3843.00, 3840.98]
QoE = [1265.93, 3850.92, 3850.02]

bitRate     3843.000000
QoE         3850.921166
Diff           0.000000

buffTime       0.000000
bitRate      537.990333
QoE         1265.932015
Diff           1.153656

# 100K
buffTime       2.620000
bitRate      894.326908
QoE         1661.082571
Diff          13.927267

# 1M
buffTime       0.000000
bitRate     3840.988200
QoE         3850.026317
Diff           0.040675

# Throughput 20M
bitrate = [538.31, 3843.00, 3843.00]
QoE = [1266.80, 3850.92, 3850.92]

# Throughput 10K
buffTime       0.000000
bitRate      538.311467
QoE         1266.802671
Diff           1.178494, 

# 100K
bitRate     3843.000000
QoE         3850.921166
Diff           0.000000

buffTime       0.000000
bitRate     1060.246867
QoE         2059.615942
Diff           6.535111

# 1M
buffTime       0.000000
bitRate     3843.000000
QoE         3850.921166
Diff           0.000000

In [ ]:
# Packet Loss:
# Reno 1M 3% (BOLA)
bitrate = [772.00, 619.44, 603.09]
QoE = [1759.76, 1410.24, 1377.46]

buffTime       0.000000
bitRate      619.441533
QoE         1410.246509
Diff           4.688325

# 5%
buffTime       0.000000
bitRate      603.092533
QoE         1377.461250
Diff           5.311745

# Throughput 3%
bitrate = [772.00, 641.33, 573.93]
QoE = [1759.76, 1467.47, 1306.52]

buffTime       0.000000
bitRate      641.333800
QoE         1467.479165
Diff           4.239914

# 5%
buffTime       0.000000
bitRate      573.931867
QoE         1306.526935
Diff           6.491995

# 10M (Bola) 3%
bitrate = [3843.00, 1014.06, 871.12]
QoE = [3850.92, 2065.93, 1875.67]

buffTime       0.000000
bitRate     1014.069733
QoE         2065.931824
Diff           4.935645

# 5%
buffTime       0.000000
bitRate      871.126800
QoE         1875.675913
Diff           3.459087

# Throughut 3%
bitrate = [3843.00, 1031.20, 774.77]
QoE = [3850.92, 2080.40, 1702.16]

buffTime       0.000000
bitRate     1031.200200
QoE         2080.403310
Diff           5.275265

# 5%
buffTime       0.000000
bitRate      774.774467
QoE         1702.167207
Diff           5.821699


# 20M (Bola) 3%
bitrate = [3843.00, 1005.31, 848.98]
QoE = [3850.92, 2053.16, 1814.38]

buffTime       0.000000
bitRate     1005.313133
QoE         2053.167926
Diff           5.127455

# 5%
buffTime       0.000000
bitRate      848.989333
QoE         1814.385388
Diff           4.582899

# Throughput 3%
bitrate = [3843.00, 1035.87, 932.63]
QoE = [3850.92, 2102.31, 1981.39]

buffTime       0.000000
bitRate     1035.873267
QoE         2102.319920
Diff           4.339196

# 5%
buffTime       0.000000
bitRate      932.639400
QoE         1981.395496
Diff           1.506497

In [ ]:
# Packet Loss:
# Cubic 1M 3% (BOLA)

bitrate = [772.00, 627.77, 559.55]
QoE = [1759.76, 1431.40, 1292.60]

buffTime       0.000000
bitRate      627.776533
QoE         1431.402287
Diff           4.760990

# 5%
buffTime       0.000000
bitRate      559.551433
QoE         1292.608378
Diff           5.117240

# Throughput 3%
bitrate = [772.00, 690.36, 537.84]
QoE = [1759.76, 1573.18, 1242.55]

buffTime       0.000000
bitRate      690.361933
QoE         1573.180535
Diff           2.345351

# 5%
buffTime       0.000000
bitRate      537.848467
QoE         1242.551598
Diff           5.477710

# Bola 10M 3%
bitrate = [3843.00, 775.94, 545.40]
QoE = [3850.92, 1685.60, 1190.62]

buffTime       0.000000
bitRate      775.940133
QoE         1685.608571
Diff           7.363157

# 5%
buffTime       0.000000
bitRate      545.400467
QoE         1190.625284
Diff           7.627253


# Throughput 3%
bitrate = [3843.00, 763.54, 580.54]
QoE = [3850.92, 1688.57, 1321.96]

buffTime       0.000000
bitRate      763.546800
QoE         1688.575781
Diff           5.875612

# 5%
buffTime       0.000000
bitRate      580.545200
QoE         1321.961085
Diff           6.535429

# 20M 3% (Bola)
bitrate = [3843.00, 793.99, 583.52]
QoE = [3850.92, 1701.75, 1292.16]

buffTime       0.000000
bitRate      793.991067
QoE         1701.759404
Diff           5.334774

# 5%
buffTime       0.000000
bitRate      583.520867
QoE         1292.167468
Diff           7.450301

# Throughput 3%
bitrate = [3843.00, 814.83, 610.69]
QoE = [3850.92, 1759.41, 1361.57]

buffTime       0.000000
bitRate      814.838067
QoE         1759.416468
Diff           5.589635

# 5%
buffTime       0.000000
bitRate      610.698800
QoE         1361.579028
Diff           6.211292

In [16]:
# Packet Loss:
# BBRv2 1M 3% (BOLA)
bitrate = [772.00, 690.78, 618.80]
QoE = [1759.76, 1575.32, 1413.71]

buffTime       0.000000
bitRate      690.788444
QoE         1575.320272
Diff           2.571448

# 5%
buffTime       0.000000
bitRate      618.805533
QoE         1413.719058
Diff           5.360804

# Throughput 3%
bitrate = [772.00, 727.15, 539.80]
QoE = [1759.76, 1660.33, 1237.49]

buffTime       0.000000
bitRate      727.154267
QoE         1660.333845
Diff           1.294751

# 5%
buffTime       0.000000
bitRate      539.800200
QoE         1237.492609
Diff           6.566507

# 10M (BOLA) 3%
bitrate = [3843.00, 2726.44, 1601.22]
QoE = [3850.92, 3345.66, 2650.10]

buffTime       0.000000
bitRate     2726.441333
QoE         3345.665079
Diff           1.303088

# 5%
buffTime       0.000000
bitRate     1601.222067
QoE         2650.107075
Diff           1.869972

# Throughput 3%
bitrate = [3843.00, 2383.27, 1547.34]
QoE = [3850.92, 3157.32, 2587.03]

buffTime       0.000000
bitRate     2383.279467
QoE         3157.326354
Diff           2.293509

# 5%
buffTime       0.000000
bitRate     1547.348133
QoE         2587.034028
Diff           2.122086

# 20M Bola(3%)
bitrate = [3843.00, 2745.41, 1576.74]
QoE = [3850.92, 3341.46, 2580.03]

buffTime       0.000000
bitRate     2745.416200
QoE         3341.463453
Diff           1.136837

# 5%
buffTime       0.000000
bitRate     1576.746667
QoE         2580.037306
Diff           2.752416

# Throughput 3%
bitrate = [3843.00, 2549.11, 1470.14]
QoE = [3850.92, 3244.77, 2476.10]

buffTime       0.000000
bitRate     2549.115533
QoE         3244.778672
Diff           1.914324

# 5%
buffTime       0.000000
bitRate     1470.145000
QoE         2476.100469
Diff           3.807275


In [ ]:
# Packet Loss:
# BBR 1M 3% (BOLA)
bitrate = [772.00, 638.51, 559.92]
QoE = [1759.76, 1461.16, 1276.28]

buffTime       0.000000
bitRate      638.510733
QoE         1461.162373
Diff           3.975151
# 5%
buffTime       0.000000
bitRate      559.921267
QoE         1276.288750
Diff           5.226306

#(Throughput)
#3%

bitrate = [772.00, 653.59, 558.81]
QoE = [1759.76, 1494.48, 1279.24]

buffTime       0.000000
bitRate      645.576800
QoE         1474.202238
Diff           4.320509
#5%
buffTime       0.000000
bitRate      558.815083
QoE         1279.240262
Diff           6.507120(One file missing)


# BBR 10M 3% (BOLA)
bitrate = [3843.00, 3840.89, 3834.85]
QoE = [3850.92, 3849.98, 3847.30]

buffTime       0.000000
bitRate     3840.892400
QoE         3849.985642
Diff           0.040675

#5%
buffTime       0.000000
bitRate     3834.857000
QoE         3847.301096
Diff           0.162700

# Throughput 3%
bitrate = [3843.00, 3840.70, 3722.90]
QoE = [3850.92, 3849.90, 3209.54]

buffTime       0.000000
bitRate     3840.700800
QoE         3849.904293
Diff           0.040675

# 5%
buffTime       0.000000
bitRate     3722.904292
QoE         3209.541804
Diff           0.240057


# BBR 20M 3%(BOLA)
bitrate = [3840.98, 3813.11, 3836.96]
QoE = [3850.02, 3838.02, 3848.23]

buffTime       0.000000
bitRate     3813.110400
QoE         3838.027207
Diff           0.203375

# 5%
buffTime       0.000000
bitRate     3836.964600
QoE         3848.236620
Diff           0.122025


# Throughput 3%
bitrate = [3843.00, 3839.16, 3838.49]
QoE = [3850.92, 3849.21, 3848.92]

buffTime       0.000000
bitRate     3839.168000
QoE         3849.212818
Diff           0.081350

# 5%
buffTime       0.000000
bitRate     3838.497400
QoE         3848.928094
Diff           0.081350

In [ ]:
#Delay:
# Reno 1M 200ms (Bola)
bitrate = [, 691.61, 523.76]
QoE = [, 1583.18, 1196.87]

buffTime       0.000000
bitRate      691.619267
QoE         1583.184230
Diff           2.970001

# 300ms
buffTime       0.000000
bitRate      523.765933
QoE         1196.872324
Diff           7.340195

# Throughput 200ms
bitrate = [, 632.78, 514.37]
QoE = [, 1445.60, 1172.08]

buffTime       0.000000
bitRate      632.789867
QoE         1445.605760
Diff           4.510756

# 300ms
buffTime       0.000000
bitRate      514.378467
QoE         1172.087486
Diff           6.959691



# 10M (Bola) 200ms
bitrate = [, 1955.56, 1130.37]
QoE = [3850.92, 2926.88, 2224.03]

buffTime       0.000000
bitRate     1955.568000
QoE         2926.884524
Diff           2.653528

# 300ms
buffTime       0.000000
bitRate     1130.379067
QoE         2224.038343
Diff           3.001739

# Throughut 200ms
bitrate = [, 2103.08, 1045.48]
QoE = [, 2926.88, 3029.7]

buffTime       0.000000
bitRate     2103.088000
QoE         3029.708733
Diff           1.769019

# 300ms
buffTime       0.000000
bitRate     1045.489400
QoE         2125.326677
Diff           3.888519


# 20M (Bola) 200ms
bitrate = [, 2133.53, 1131.97]
QoE = [3850.92, 3041.19, 2226.79]

buffTime       0.000000
bitRate     2133.532067
QoE         3041.199342
Diff           1.782866

# 300ms
buffTime       0.000000
bitRate     1131.974133
QoE         2226.791594
Diff           3.024792


# Throughput 200ms
bitrate = [, 2255.47, 1099.28]
QoE = [, 3135.89, 2185.61]

buffTime       0.000000
bitRate     2255.472000
QoE         3135.894077
Diff           0.884509

# 300ms
buffTime       0.000000
bitRate     1099.286933
QoE         2185.610996
Diff           3.750540

In [ ]:
# #Cubic Delay 1M, 200ms:
# Bola
bitrate = [, 689.32, 515.07]
QoE = [, 1577.55, 1173.57]

bitRate      689.327933
QoE         1577.550188
Diff           2.730074

# 300ms
bitRate      515.073667
QoE         1173.575665
Diff           7.381411

#Throughput 200ms
bitrate = [, 722.59, 1647.52]
QoE = [, 1647.52, 1149.13]

bitRate      722.596200
QoE         1647.520757
Diff           1.385648

#300ms
bitRate      505.523800
QoE         1149.137832
Diff           6.965771

# 10M Bola 200ms
bitrate = [, 1948.75, 1215.83]
QoE = [3850.92, 2918.23, 2321.24]

bitRate     1948.758533
QoE         2918.233206
Diff           2.914024

# 300ms
bitRate     1215.832933
QoE         2321.244044
Diff           2.327032

# Throughput 200ms
bitrate = [, 1811.44, 1177.88]
QoE = [, 2826.40, 2273.47]

bitRate     1811.440000
QoE         2826.404266
Diff           3.538037

# 300ms
bitRate     1177.889111
QoE         2273.476079
Diff           2.927510

# 20M Bola 200ms
bitrate = [, 2094.33, 1133.72]
QoE = [3850.92, 3019.60, 2228.41]

bitRate     2094.331267
QoE         3019.607214
Diff           1.785952

# 300ms
bitRate     1133.729933
QoE         2228.415306
Diff           3.102320

# Throughput 200ms
bitrate = [, 2108.20, 1126.76]
QoE = [, 3032.66, 2216.40]

bitRate     2108.205733
QoE         3032.663732
Diff           1.909531

# 300ms
bitRate     1126.769733
QoE         2216.407055
Diff           3.217448

In [ ]:
# Delay with bandwidth 1M.
# Bola BBRv2
bitrate = [, 589.19, 536.95]
QoE = [, 1345.90, 1228.66]

#Delay:
# BBRv2 1M 200ms (Bola)
buffTime       0.000000
bitRate      589.191133
QoE         1345.901748
Diff           6.527138

# 300ms
buffTime       0.000000
bitRate      536.956533
QoE         1228.668223
Diff           7.222603

# Throughput
bitrate = [, 544.31, 517.21]
QoE = [, 1247.49, 1175.17]

# Throughput 200ms
buffTime       0.000000
bitRate      544.316800
QoE         1247.497857
Diff           7.352340

# 300ms
buffTime       0.000000
bitRate      517.215333
QoE         1175.179469
Diff           7.681320


# Delay with bandwidth 10M.
# Bola
bitrate = [, 2279.51, 1392.56]
QoE = [3850.92, 3141.26, 2515.51]
# 10M 200ms (Bola)
buffTime       0.000000
bitRate     2279.510333
QoE         3141.263080
Diff           0.908018

# 300ms
buffTime       0.000000
bitRate     1392.564000
QoE         2515.510336
Diff           1.037742

# Throughput
bitrate = [, 2345.41, 1387.32]
QoE = [, 3191.04, 2510.78]
# Throughput 200ms
buffTime       0.000000
bitRate     2345.417333
QoE         3191.041146
Diff           0.369476

# 300ms
buffTime       0.000000
bitRate     1387.322800
QoE         2510.782668
Diff           0.903671

# Delay with bandwidth 20M.
# Bola
bitrate = [, 2321.59, 1395.49]
QoE = [3830.76, 3177.95, 2519.60]
# 20M (Bola) 200ms
buffTime       0.000000
bitRate     2321.596467
QoE         3177.956631
Diff           0.715444

# 300ms
buffTime       0.000000
bitRate     1395.498800
QoE         2519.603454
Diff           0.907307


# Throughput
bitrate = [, 2296.75, 1397.43]
QoE = [, 3153.21, 2523.08]
# Throughput 200ms
buffTime       0.000000
bitRate     2296.757600
QoE         3153.215575
Diff           0.731117

# 300ms
buffTime       0.000000
bitRate     1397.435400
QoE         2523.089671
Diff           0.748600


In [1]:
#New Using only NetEm Result:
#Delay: BBR 1M Bola
# 200ms
bitRate      587.549133
QoE         1343.138399
Diff           6.126026

# 300ms
bitRate      527.709667
QoE         1209.558866
Diff           6.814894

# Throughput 200ms
bitRate      673.172667
QoE         1539.936853
Diff           3.074814

# 300ms
bitRate      504.001444
QoE         1148.883164
Diff           7.303909

# 10M Bola 200ms


In [ ]:
Globcomm Result:
# 3%: (Bola)
QoE [5537.59, 5318.86, 5346.27, 4891.95, 5241.47]
QoE [6113.58, 5091.75, 4654.66, 5812.05, 5086.46]
QoE [5682.73, 5953.66, 5992.98, 5568.86, 5909.00]
QoE [5876.71, 5889.56, 6152.79, 6012.00, 5917.44]
# 7%: (Bola)
QoE [3098.53, 3393.65, 4004.66, 2734.79, 3116.62]
QoE [3046.30, 3796.91, 2582.64, 3816.91, 2666.78]
QoE [2515.76, 2135.89, 2600.43, 2552.08, 2720.52]
QoE [2107.92, 3369.93, 2994.91, 3499.68, 3130.24]

# 3%:(Throughput)
QoE [6168.97, 5083.54, 5456.49, 5401.88, 4940.35]
QoE [6029.10, 5269.95, 2742.60, 5525.77, 5474.23]
QoE [5258.66, 5859.29, 6071.55, 5505.10, 5303.30]
QoE [5949.24, 6188.02, 3933.34, 6134.77, 6193.35]
# 7%:(Throughput)
QoE [4017.78, 3861.24, 3040.52, 3431.00, 3582.50]
QoE [2346.11, 3900.89, 4076.10, 3240.60, 3168.21]
QoE [3387.65, 2876.39, 2732.91, 2945.88, 2699.49]
QoE [2136.58, 4262.44, 3933.34, 4147.23, 1987.64]

In [209]:
#Bola Packet Loss variation: 79.34, 79.26
BBR:
    
buffTime    [1.880000, 24.12, 51.18, 79.34, 95.88]
bitRate     [1148.074418, 853.44, 744.93, 562.32, 420.88]
QoE         [6700.252151, 5267.23, 4673.09, 3269.65, 1958.72]

BBRv2:    
buffTime     [3.200000, 28.86, 48.57, 79.26, 98.48]
bitRate      [1075.597778, 865.37, 707.38, 545.80, 348.32]
QoE          [6440.738631, 5351.70, 4429.59, 3181.91, 1310.61]

Cubic:
buffTime    [1.380000, 13.22, 56.28, 105.52, 158.56]
bitRate     [1295.777778, 937.47, 575.79, 477.21, 320.67]
QoE         [7188.054729, 5821.45, 3077.58, 2504.93, 919.42]


Reno:
buffTime    [2.940000, 8.74, 40.08, 100.44, 148.12]
bitRate     [1255.508889, 964.83, 712.64, 562.88, 352.96]
QoE         [7032.114732, 5969.70, 4134.15, 3020.54, 1276.45]


6682.3225

In [ ]:
####### Throughput Packet Loss #######82.52, 66.44, 82.72
BBR:
buffTime    [2.960000, 25.44, 49.86, 51.42, 95.66]
bitRate     [1157.680000, 872.46, 830.65, 588.82, 420.06]
QoE         [6763.675103, 5410.25, 5069.37, 3586.61, 2706.21]

BBRv2:
buffTime    [3.840000, 12.96, 40.04, 59.80, 115.54]
bitRate     [1094.668889, 796.92, 819.74, 546.10, 398.57]
QoE         [6496.339292, 5008.33, 5004.66, 3346.38, 1721.49]

Cubic:
buffTime    [1.520000, 22.72, 63.08, 100.36, 197.84]
bitRate     [1224.113333, 871.14, 660.48, 511.39, 313.84]
QoE         [6981.154830, 5599.58, 3985.86, 2928.46, 704.35]

Reno:
buffTime    [3.680000, 17.32, 45.14, 64.54, 126.16]
bitRate     [1348.053333, 900.92, 863.03, 553.95, 376.55]
QoE         [7374.953166, 5679.74, 5252.03, 3293.44 1738.70]


In [ ]:
##### Bandwidth varition with 1% packet Loss ###########
Bola
BBR:
buffTime [99.400, 7.6400, 24.360000, 7.36000, 1.660000]  
bitRate  [200.000, 200.0000, 512.92, 778.43, 1095.38]
QoE      [-264.404, -20.32, 3460.85, 5193.81, 6542.39]

BBRv2:
buffTime [219.80, 30.10, 2.72, 9.34, 3.0]
bitRate [200.00, 200.00, 510.46, 641.63, 1065.74]
QoE     [-584.668, -80.06, 3417.68, 4010.83, 6443.50]


Cubic:   
buffTime  [7.700, 2.34, 4.16, 1.75, 1.10]
bitRate   [200.000,211.04, 572.53, 786.56, 1242.28]
QoE       [-20.482, 137.60, 4049.13, 5311.98,6990.23]

Reno:
buffTime  [15.50, 0.02, 3.00, 1.66, 2.4]
bitRate   [200.00, 200.00, 546.57, 794.28, 1255.50]
QoE       [-41.23,-0.05, 3829.36, 5367.75,  7032.11]


In [ ]:
Throughput
BBR:
buffTime    [194.050, 28.12, 34.62, 9.48, 2.30]
bitRate     [200.000, 200, 506.10, 754.00, 992.03]
QoE         [-516.173, -63.16, 3391.97.91, 5047.96, 6158.70]

BBRv2:
buffTime    [127.200, 20.20, 40.28, 16.68, 8.10]
bitRate     [200.000, 201.12, 519.88, 747.67, 978.43]
QoE         [-338.352, -38.51, 3478.13, 5025.27, 6166.90]

Cubic:
buffTime     [19.50, 1.62, 4.88, 2.18, 0.40]
bitRate     200.00, 207.62, 523.63, 791.49, 1008.96]
QoE         [-51.87, 94.54, 3647.59, 5357.55, 6259.33]

Reno:
buffTime     [21.400, 0.00, 5.61, 1.66, 1.60]
bitRate     [200.000, 202.97, 512.62, 785.56, 1351.30]
QoE         [-56.924, 38.41, 3427.94, 5318.11, 7380.89]



In [ ]:
#Bola Packet Loss variation
BBR:
    
buffTime    [1.880000, 15.78, 36.46, 51.10, 101.08]
bitRate     [1148.074418, 830.50, 729.36, 469.31, 386.51]
QoE         [6700.252151, 5213.85, 4475.24, 2433.39, 1594.85]

BBRv2:
    
buffTime     [3.200000, 19.06, 40.08, 57.60, 98.48]
bitRate      [1075.597778, 853.35, 653.16, 619.02, 348.32]
QoE          [6440.738631, 5313.77, 4041.15, 3759.01, 1310.61]

Cubic:
buffTime    [1.380000, 16.62, 49.38, 72.50, 158.56]
bitRate     [1295.777778, 909.88, 735.73, 441.65, 320.67]
QoE         [7188.054729, 5312.10, 4539.76, 2189.32, 919.42]

Reno:
buffTime    [2.940000, 8.74, 40.08, 65.17, 117.98]
bitRate     [1255.508889, 1079.19, 832.08, 619.77, 386.95]
QoE         [7032.114732, 6381.69, 5195.14, 3636.06, 1764.13]


In [ ]:
####### Throughput Packet Loss #######
BBR:
buffTime    [2.960000, 21.88, 59.86, 51.42, 95.66]
bitRate     [1157.680000, 953.45, 697.93, 514.78, 361.88]
QoE         [6763.675103, 5875.74, 4357.30, 3017.76, 1528.66]

BBRv2:
buffTime    [3.840000, 20.86, 40.04, 69.80, 106.58]
bitRate     [1094.668889, 919.30, 739.52, 577.55, 420.06]
QoE         [6496.339292, 5776.21, 4772.99, 3573.72, 2061.11]

Cubic:
buffTime    [1.520000, 11.16, 63.08, 118.36, 122.26]
bitRate     [1224.113333, 976.49, 736.85, 509.57, 282.10]
QoE         [6981.154830, 6073.47, 4664.68, 2886.47, 594.40]

Reno:
buffTime    [3.680000, 11.17, 31.54, 64.54, 126.16]
bitRate     [1348.053333, 1021.66, 933.96, 657.45, 476.55]
QoE         [7374.953166, 6212.54, 5683.97, 3904.29, 2560.02]
